In [ ]:
!pip install torch-geometric

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
#Import
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch.nn import Parameter
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax
import torch.optim as optim
from torch_geometric.nn.conv import feast_conv


In [2]:
!nvcc --version

if torch.cuda.is_available():
    device_name = torch.device("cuda")
else:
    device_name = torch.device('cpu')

print("Using {}.".format(device_name))

Using cpu.


'nvcc' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
#data

import numpy as np
import torch
import random
from torch_geometric.data import Data, DataLoader

#Import data function
def get_data(filepath):
    data = np.loadtxt(filepath)
    x = data[:, :20]
    z = data[:, 20:40]
    m = data[:, 40:60]

    dataX = []
    for i in range(x.shape[0]):
        dataXs = np.column_stack((np.arange(20), x[i], z[i]))
        dataX.append(dataXs)

    dataY = m
    zipped = list(zip(dataX, dataY))
    np.random.shuffle(zipped)
    dataX, dataY = zip(*zipped)

    dataX = np.array(dataX)
    dataY = np.array(dataY)

    dataEdgeIndex = np.column_stack((np.arange(20)[:-1], np.arange(20)[1:]))
    dataEdgeIndex = np.vstack((dataEdgeIndex, dataEdgeIndex[:, ::-1])).T

    dataset = [Data(x=torch.from_numpy(x).float(), edge_index=torch.from_numpy(dataEdgeIndex).long(), y=torch.from_numpy(y).float()) for x, y in zip(dataX, dataY)]

    return dataset


#Getting data
dataset = get_data("C:\\Users\\eliasak\\OneDrive - NTNU\\Master thesis\\04_Elias\\ML2\\masters2023\\ELIAS\\data4.txt")
print(len(dataset))
#Splitting dataset
datasetRun = dataset[:int(len(dataset)*0.90)]
datasetTest = dataset[int(len(dataset)*0.90):]

train_loader = datasetRun[:int(len(datasetRun)*0.75)]
test_loader = datasetRun[int(len(datasetRun)*0.75):]



14746


In [24]:
#Data Checks

print(len(train_loader))
print(len(test_loader))

# one data object from train_loader:
graph = train_loader[55]
print(graph.x.shape)
print(graph.edge_index.shape)
print(graph.y.shape)
print(graph.x, graph.y)

9953
3318
torch.Size([20, 3])
torch.Size([2, 38])
torch.Size([20])
tensor([[  0.0000,   0.0000,   0.0000],
        [  1.0000,   0.8950,   8.7603],
        [  2.0000,   3.2016,  17.2588],
        [  3.0000,   6.8586,  25.2695],
        [  4.0000,  11.7686,  32.5794],
        [  5.0000,  17.8012,  38.9944],
        [  6.0000,  24.7961,  44.3438],
        [  7.0000,  32.5673,  48.4855],
        [  8.0000,  40.9082,  51.3093],
        [  9.0000,  49.5970,  52.7402],
        [ 10.0000,  58.4030,  52.7402],
        [ 11.0000,  67.0919,  51.3093],
        [ 12.0000,  75.4327,  48.4855],
        [ 13.0000,  83.2039,  44.3438],
        [ 14.0000,  90.1988,  38.9944],
        [ 15.0000,  96.2314,  32.5794],
        [ 16.0000, 101.1414,  25.2695],
        [ 17.0000, 104.7984,  17.2588],
        [ 18.0000, 107.1050,   8.7603],
        [ 19.0000, 108.0000,   0.0000]]) tensor([ 0.2616, -0.0493,  0.0833,  0.1421,  0.1399,  0.0938,  0.0240, -0.0498,
        -0.1099, -0.1434, -0.1434, -0.1099, -0.0498,

In [6]:
# ArchNN

class ArchNN(torch.nn.Module):
    def __init__(self, in_channels, num_classes, heads, t_inv = True, l1_lambda=0.001):
        super(ArchNN, self).__init__()
        self.fc0 = nn.Linear(in_channels, 16) 
        #self.bn0 = nn.BatchNorm1d(16)
        self.conv1 = feast_conv.FeaStConv(16, 32, heads=heads, t_inv=t_inv)
        self.conv2 = feast_conv.FeaStConv(32, 64, heads=heads, t_inv=t_inv)
        self.conv3 = feast_conv.FeaStConv(64, 128, heads=heads, t_inv=t_inv)
        #self.conv4 = feast_conv.FeaStConv(128, 256, heads=heads, t_inv=t_inv)
        self.fc1 = nn.Linear(128, 256)
        #self.bn1 = nn.BatchNorm1d(256) 
        self.fc2 = nn.Linear(256, 1)

        self.reset_parameters()
        self.l1_lambda = l1_lambda

    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()
        self.conv3.reset_parameters()
        #self.conv4.reset_parameters()

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.elu(self.fc0(x))
        x = F.elu(self.conv1(x, edge_index))
        x = F.elu(self.conv2(x, edge_index))
        x = F.elu(self.conv3(x, edge_index))
        #x = F.elu(self.conv4(x, edge_index))
        x = F.elu(self.fc1(x))
        #x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        #F.log_softmax(x, dijm=1)
        x = torch.squeeze(x, dim=1)
        return x

    """def compute_loss(self, outputs, targets):
      # Define the loss function
      mse_loss = nn.MSELoss()
      l1_loss = torch.tensor(0.0, requires_grad=True).to(outputs.device)  # initialize L1 loss to 0
      for name, param in self.named_parameters():
          if 'weight' in name:  # apply L1 regularization only to weight parameters
              l1_loss += torch.norm(param, p=1)  # compute element-wise L1 norm
      loss = mse_loss(outputs, targets) + self.l1_lambda * l1_loss  # add L1 regularization term to the loss
      return loss"""

In [7]:

import time
import torch
import torch.nn.functional as F


def print_info(info):
    message = ('Epoch: {}/{}, Duration: {:.3f}s,'
               'Train Loss: {:.4f}, Test Loss:{:.4f}').format(
                   info['current_epoch'], info['epochs'], info['t_duration'],
                   info['train_loss'], info['test_loss'])
    print(message)


def run(model, train_loader, test_loader, num_nodes, epochs, optimizer, device):

    for epoch in range(1, epochs + 1):
        #random.shuffle(train_loader) #added this this see if it helps, tihihi
        #random.shuffle(test_loader)  #added this this see if it helps, tihihi
        t = time.time()
        train_loss = train(model, train_loader, optimizer, device)
        t_duration = time.time() - t
        test_loss = test(model, test_loader, num_nodes, device)
        eval_info = {
            'train_loss': train_loss,
            'test_loss': test_loss,
            'current_epoch': epoch,
            'epochs': epochs,
            't_duration': t_duration
        }

        print_info(eval_info)


def train(model, train_loader, optimizer, device):
    model.train()

    total_loss = 0
    for idx, data in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data.to(device))
        #log_probs = F.log_softmax(output, dim=1).double()
        #print(log_probs.dtype)
        #print(data.y.dtype)
        loss = F.mse_loss(output, data.y.to(device))

        #for i in range(data.num_nodes):
        #    loss = F.mse_loss(output[i], data.y[i].to(device))
        #    loss.backward(retain_graph=True)

        #loss = model.compute_loss(output, data.y)  # compute loss with L1 regularization
        #loss = F.nll_loss(log_probs, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)


def test(model, test_loader, num_nodes, device):
    model.eval()
    correct = 0
    total_loss = 0
    n_graphs = 0
    with torch.no_grad():
        for idx, data in enumerate(test_loader):
            out = model(data.to(device))
            total_loss += F.mse_loss(out, data.y.to(device)).item()
            #pred = out.max(1)[1]
            #correct += pred.eq(data.y).sum().item()
            #n_graphs += data.num_graphs
    return total_loss / len(test_loader)


In [11]:
#runner
num_nodes = train_loader[0].x.shape[0]
num_features = train_loader[0].x.shape[1]

model = ArchNN(num_features, num_nodes, heads=8).to(device_name)

optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.0005)


run(model, train_loader, test_loader, num_nodes, 10, optimizer, device_name)


torch.save(model, "1904.pt")

Epoch: 1/10, Duration: 101.472s,Train Loss: 0.1084, Test Loss:0.0914
Epoch: 2/10, Duration: 101.812s,Train Loss: 0.0719, Test Loss:0.0757
Epoch: 3/10, Duration: 104.178s,Train Loss: 0.0384, Test Loss:0.0176
Epoch: 4/10, Duration: 97.836s,Train Loss: 0.0130, Test Loss:0.0111
Epoch: 5/10, Duration: 97.833s,Train Loss: 0.0103, Test Loss:0.0096
Epoch: 6/10, Duration: 103.823s,Train Loss: 0.0089, Test Loss:0.0081
Epoch: 7/10, Duration: 109.383s,Train Loss: 0.0080, Test Loss:0.0080
Epoch: 8/10, Duration: 104.779s,Train Loss: 0.0073, Test Loss:0.0077
Epoch: 9/10, Duration: 112.935s,Train Loss: 0.0067, Test Loss:0.0073
Epoch: 10/10, Duration: 106.730s,Train Loss: 0.0063, Test Loss:0.0067


In [26]:
#More epochs per favoure?
num_nodes = train_loader[0].x.shape[0]
num_features = train_loader[0].x.shape[1]
run(model, train_loader, test_loader, num_nodes, 5, optimizer, device_name)

Epoch: 1/5, Duration: 159.629s,Train Loss: 0.0059, Test Loss:0.0065
Epoch: 2/5, Duration: 222.661s,Train Loss: 0.0056, Test Loss:0.0063
Epoch: 3/5, Duration: 157.188s,Train Loss: 0.0053, Test Loss:0.0059
Epoch: 4/5, Duration: 157.807s,Train Loss: 0.0052, Test Loss:0.0072
Epoch: 5/5, Duration: 161.060s,Train Loss: 0.0051, Test Loss:0.0069


In [27]:
#Save?
ModelName = "1904Arch"
torch.save(model.state_dict(), ModelName)

#Load:
'''
model = ArchNN(*args, **kwargs)
model.load_state_dict(torch.load(ModelName))
model.eval()
'''

'\nmodel = ArchNN(*args, **kwargs)\nmodel.load_state_dict(torch.load(ModelName))\nmodel.eval()\n'

In [16]:
#Test for new dataline

def testData(line):
    real = datasetTest[line].y
    
    out = model(datasetTest[line].to(device_name))
    Bool = ''
    print('{:<10}{:<10}{:<10}{:<10}'.format('real', 'out', '%error', 'Sign'))
    for i in range(len(out)):
        error = ((out[i].item() - real[i].item())/ out[i] )*100
        if torch.sign(real[i]) == torch.sign(out[i]):
            Bool = 'True'
        else:
            Bool = 'False'
        real_formatted = '{:<10.3f}'.format(round(real[i].item(),5))
        out_formatted = '{:<10.3f}'.format(round(out[i].item(),5))
        error_formatted = '{:<10.2f}'.format(error)
        list_formatted = [real_formatted, out_formatted, error_formatted, Bool]
        print('{:<10}{:<10}{:<10}{:<10}'.format(*list_formatted))

#Test
testData(0)

real      out       %error    Sign      
0.057     0.070     17.53     True      
0.007     0.025     73.73     True      
0.042     0.055     23.38     True      
0.054     0.061     12.25     True      
0.046     0.039     -15.85    True      
0.025     0.013     -97.80    True      
-0.002    -0.003    10.08     True      
-0.029    -0.023    -28.16    True      
-0.051    -0.030    -68.64    True      
-0.062    -0.045    -37.66    True      
-0.062    -0.044    -40.72    True      
-0.051    -0.041    -23.85    True      
-0.029    -0.025    -16.79    True      
-0.002    0.006     141.80    False     
0.025     0.038     35.46     True      
0.046     0.073     37.02     True      
0.054     0.083     35.18     True      
0.042     0.081     47.39     True      
0.007     0.067     90.02     True      
-0.057    0.026     318.98    False     


In [13]:
#Test for new dataset
def testFinal(model, indata, device):
    model.eval()
    #correct = 0
   # total_loss = 0
   # n_graphs = 0
    errorAVG = 0
    with torch.no_grad():
        for id, data in enumerate(indata):
          out = model(data.to(device))
          real = data.y.to(device)
          errorList = []
          error = 0
          for i in range(len(out)):
            errorList.append(round(((out[i].item() - real[i].item() ) / out[i].item())*100, 2))
            error += abs(((out[i].item() - real[i].item() ) / out[i].item())*100)

          errorAVG += round(error/len(out),2)
          print(errorList)
          print('Avarage error: '+ str(round(error/len(out),2))+'%')
          print('Mean Square: '+ str(round(F.mse_loss(out, data.y.to(device)).item(),2))+'\n')
    
    print('The avarage error is ' + str(errorAVG/len(indata))+'%')


#Load test data set, if this is not done from beginning!
#datasetTest = get_data("/content/drive/MyDrive/datasets/data1test.txt")

#Run testFinal
testFinal(model, datasetTest,  device_name)


[17.53, 73.73, 23.38, 12.25, -15.85, -97.8, 10.08, -28.16, -68.64, -37.66, -40.72, -23.85, -16.79, 141.8, 35.46, 37.02, 35.18, 47.39, 90.02, 318.98]
Avarage error: 58.61%
Mean Square: 0.0

[2.38, 109.02, 17.58, 0.18, -33.5, -59.85, 143.79, -20.74, -24.26, -17.0, -18.22, -16.0, -28.24, 88.76, 22.88, 21.48, 19.57, 16.24, 105.33, -199.56]
Avarage error: 48.23%
Mean Square: 0.0

[-1.98, 124.12, -15.39, -21.19, -20.26, -10.53, 87.4, -3.56, 3.44, 2.64, -6.98, -35.0, -111.02, 98.49, 26.01, 0.56, 13.73, 0.12, 110.78, -5.35]
Avarage error: 34.93%
Mean Square: 0.0

[14.67, -4267.14, 34.51, 23.05, 25.64, 25.9, 57.01, -133.58, -41.13, -38.43, -57.58, -40.53, -8064.69, 62.09, 30.29, 16.31, 8.72, 17.83, -30.27, -19.41]
Avarage error: 650.44%
Mean Square: 0.01

[-52.6, 21.82, 33.19, 42.5, 67.16, 118.25, 263.85, -2297.57, -255.91, -193.84, -244.64, -422.59, 1649.74, 207.54, 111.54, 77.06, 48.51, 41.68, 13.25, 25.71]
Avarage error: 309.45%
Mean Square: 0.0

[150.38, 48.83, 55.24, 69.01, 89.15, 117.26, 

In [ ]:
#test runner
num_nodes = train_loader[0].x.shape[0]
num_features = train_loader[0].x.shape[1]

modelTest = ArchNN(num_features, num_nodes, heads=8).to(device_name)

optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.005)


run(modelTest, train_loader, test_loader, num_nodes, 5, optimizer, device_name)